In [144]:
from __future__ import print_function
import sys
import IPython
print('IPython:', IPython.__version__)
import numpy
print('numpy:', numpy.__version__)
import pandas
print('pandas:', pandas.__version__)
import sklearn
print('scikit-learn:', sklearn.__version__)
print('Python version:', sys.version)

IPython: 7.34.0
numpy: 1.25.2
pandas: 1.5.3
scikit-learn: 1.2.2
Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/DATA QUALITY MANAGEMENT')

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/DATA QUALITY MANAGEMENT

/content/drive/My Drive/Colab Notebooks/DATA QUALITY MANAGEMENT


In [ ]:
import pandas as pd
import numpy as np
import random
import string
import re

In [ ]:
def generate_valid_email():
    recipient_name = generate_recipient_name()
    domain_name = generate_domain_name()
    top_level_domain = generate_top_level_domain()
    return f"{recipient_name}@{domain_name}.{top_level_domain}"

def generate_recipient_name():
    recipient_length = random.randint(1, 20)
    recipient_name = ''.join(random.choices(string.ascii_letters + string.digits + ".-_", k=recipient_length))
    # Ensure no consecutive special characters
    recipient_name = ''.join(random.sample(recipient_name, len(recipient_name)))
    return recipient_name

def generate_domain_name():
    domain_length = random.randint(1, 30)
    domain_name = ''.join(random.choices(string.ascii_letters + string.digits + "-", k=domain_length))
    return domain_name

def generate_top_level_domain():
    top_level_domains = ['com', 'net', 'org', 'co.uk', 'edu', 'gov', 'biz', 'info','co.id','my','xyz','us']
    return random.choice(top_level_domains)

def generate_invalid_email():
    # Define forbidden characters for email addresses
    forbidden_chars = ['!', '#', '$', '%', '^', '&', '*', '(', ')', '{', '}', '[', ']', ':', ';', '<', '>', '?', '/', '\\', '|', '+', '=', ' ']

    # Generate a random username
    username = ''.join(random.choices(string.ascii_letters + string.digits, k=random.randint(5, 10)))

    # Introduce a forbidden character randomly in the username
    if random.random() < 0.5:
        forbidden_char = random.choice(forbidden_chars)
        username = username[:random.randint(0, len(username))] + forbidden_char + username[random.randint(0, len(username)):]

    # Generate a random placeholder domain (single or double)
    domain_parts = []
    num_domain_parts = random.randint(1, 2)
    for _ in range(num_domain_parts):
        domain_part = ''.join(random.choices(string.ascii_lowercase, k=random.randint(5, 10)))
        domain_parts.append(domain_part)

    domain = '.'.join(domain_parts)

    # Introduce a forbidden character randomly in the domain
    if random.random() < 0.5:
        forbidden_char = random.choice(forbidden_chars)
        domain = domain[:random.randint(0, len(domain))] + forbidden_char + domain[random.randint(0, len(domain)):]

    # Concatenate username and domain to form the email address
    email = f"{username}@{domain}"

    # Introduce cases for invalid email formats
    random_num = random.random()
    if random_num < 0.1:  # 10% chance of generating email with two "@" characters
        email = f"{username}@{domain}@{domain_parts[-1]}"
    elif random_num < 0.2:  # 10% chance of generating email without "@" character
        email = ''.join(random.choices(string.ascii_lowercase, k=random.randint(5, 10)))
    elif random_num < 0.3:  # 10% chance of generating invalid email patterns with random domain
        random_domain = ''.join(random.choices(string.ascii_lowercase, k=random.randint(5, 10)))
        email = f"{username}{random_domain}"  # No "@" here

    return email


def validate_email(text):
    pattern = r'^(?!.*[._%+-]{2})[a-zA-Z0-9][a-zA-Z0-9._%+-]{0,62}@([a-zA-Z0-9-]+\.)+[a-zA-Z]{2,}(?<!\.)$'
    if re.match(pattern, text):
        return 1
    else:
        return 0

def fix_email(text):
    # Remove allowed character in the first and last string
    cleaned_String = re.sub(r'^[^\w]+|[^\w]+$', '', text)
    cleaned_string = re.sub(r"[^a-zA-Z0-9_.@-]", '', cleaned_String)
    cleaned_string = re.sub(r'\s{2,}', r' ', cleaned_string)
    matches = re.findall(r'([^\w\s])\1+', cleaned_string)
    # Remove it
    for match in matches:
        cleaned_string = cleaned_string.replace(match, match[0])

    return cleaned_string.strip()


In [ ]:
data  = []
initial_label = []
check_type = []

for _ in range(1000):
  data.append(generate_valid_email())
  initial_label.append(1)
  check_type.append('email')

for _ in range(1000):
  data.append(generate_invalid_email())
  initial_label.append(0)
  check_type.append('email')

In [ ]:
dataset = pd.DataFrame({"text" : data, 'initial_label' : initial_label, 'data_type': check_type})
dataset.to_csv('dataset/email_dataset.csv')
dataset

,text,initial_label,data_type
0,K_KuWoNLLQDa@Ygog.net,1,email
1,HYksVExW9_Th@Cz2ZfBhrjUzFFTtA2yqe4I8p.net,1,email
2,Z4CUJn_greY4UV6ZcsU1@DHqGIO4EVpv7WglCpiWopyVps...,1,email
3,Gy-28NbfxS9avw@696P1hD.co.uk,1,email
4,UVICdMaOk@r2I9zlVzFeJTa6pOQ7NeGz.com,1,email
...,...,...,...
1995,HRfCPRzb+CPRzbFFtxudhm,0,email
1996,CjiqQ3K@agntqrz[fkx,0,email
1997,cOgYp@ukvftvvund.etcpul@etcpul,0,email
1998,ZXNVOJQ8@nrx;e,0,email


In [ ]:
dataset['label'] = dataset['text'].apply(lambda x: validate_email(x))
dataset['clean'] = dataset.apply(lambda x: fix_email(str(x['text'])) if x['label'] == False else x['text'], axis=1)
dataset['check'] = dataset['clean'].apply(lambda x: validate_email(x))

dataset

,text,initial_label,data_type,label,clean,check
0,K_KuWoNLLQDa@Ygog.net,1,email,1,K_KuWoNLLQDa@Ygog.net,1
1,HYksVExW9_Th@Cz2ZfBhrjUzFFTtA2yqe4I8p.net,1,email,1,HYksVExW9_Th@Cz2ZfBhrjUzFFTtA2yqe4I8p.net,1
2,Z4CUJn_greY4UV6ZcsU1@DHqGIO4EVpv7WglCpiWopyVps...,1,email,1,Z4CUJn_greY4UV6ZcsU1@DHqGIO4EVpv7WglCpiWopyVps...,1
3,Gy-28NbfxS9avw@696P1hD.co.uk,1,email,1,Gy-28NbfxS9avw@696P1hD.co.uk,1
4,UVICdMaOk@r2I9zlVzFeJTa6pOQ7NeGz.com,1,email,1,UVICdMaOk@r2I9zlVzFeJTa6pOQ7NeGz.com,1
...,...,...,...,...,...,...
1995,HRfCPRzb+CPRzbFFtxudhm,0,email,0,HRfCPRzbCPRzbFFtxudhm,0
1996,CjiqQ3K@agntqrz[fkx,0,email,0,CjiqQ3K@agntqrzfkx,0
1997,cOgYp@ukvftvvund.etcpul@etcpul,0,email,0,cOgYp@ukvftvvund.etcpul@etcpul,0
1998,ZXNVOJQ8@nrx;e,0,email,0,ZXNVOJQ8@nrxe,0


In [ ]:
from sklearn.model_selection import train_test_split
data = dataset
# Split Feature (X) and label (Y)
X = data['text'].values
Y = data['label'].values

# Splitting data Training and Testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from joblib import dump, load

# Data dan parameter
data = dataset
max_words = 1000

# Create Model
mlp_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=max_words, analyzer='char')),
    ('clf', MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', solver='adam',
                          alpha=0.0001, random_state=42, max_iter=1000)),
])

# Train Model
history = mlp_pipeline.fit(X_train, Y_train)

# Model Evaluation
accuracy = mlp_pipeline.score(X_test, Y_test)
print("Accuracy:", accuracy)

# Classification Report
print("MLP Classification Report")
y_pred = mlp_pipeline.predict(X_test)
print(classification_report(Y_test, y_pred))

#Save models
dump(mlp_pipeline, 'model/email_mlp_model.joblib')

Accuracy: 0.92
MLP Classification Report
              precision    recall  f1-score   support

           0       0.95      0.88      0.92       197
           1       0.89      0.96      0.92       203

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400



['model/email_mlp_model.joblib']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Pipeline
lr_model_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=max_words, analyzer='char')),
    ('lr_model', LogisticRegression())
])

# Train model
lr_model_pipeline.fit(X_train, Y_train)

# Predictions
Y_train_pred = lr_model_pipeline.predict(X_train)
Y_test_pred = lr_model_pipeline.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(Y_train, Y_train_pred)
test_accuracy = accuracy_score(Y_test, Y_test_pred)
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", test_accuracy)

# Classification report
print("Classification Report Logistic Regression:")
print(classification_report(Y_test, Y_test_pred))

#Save Models
dump(lr_model_pipeline, 'model/email_lr_model.joblib')


Training Accuracy: 0.94
Validation Accuracy: 0.9275
Classification Report Logistic Regression:
              precision    recall  f1-score   support

           0       0.98      0.87      0.92       197
           1       0.89      0.98      0.93       203

    accuracy                           0.93       400
   macro avg       0.93      0.93      0.93       400
weighted avg       0.93      0.93      0.93       400



['model/email_lr_model.joblib']

In [ ]:
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Pipeline
xgb_model_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=max_words, analyzer='char')),
    ('XGB_model', xgb.XGBClassifier())
])

# Train model
xgb_model_pipeline.fit(X_train, Y_train)

# Predictions
Y_train_pred = xgb_model_pipeline.predict(X_train)
Y_test_pred = xgb_model_pipeline.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(Y_train, Y_train_pred)
test_accuracy = accuracy_score(Y_test, Y_test_pred)
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", test_accuracy)

# Classification report
print("Classification Report XGBoost:")
print(classification_report(Y_test, Y_test_pred))

#Save Models
dump(xgb_model_pipeline, 'model/email_xgb_model.joblib')


Training Accuracy: 1.0
Validation Accuracy: 0.925
Classification Report XGBoost:
              precision    recall  f1-score   support

           0       1.00      0.85      0.92       197
           1       0.87      1.00      0.93       203

    accuracy                           0.93       400
   macro avg       0.94      0.92      0.92       400
weighted avg       0.93      0.93      0.92       400



['model/email_xgb_model.joblib']

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Pipeline
svm_model_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=max_words, analyzer='char')),
    ('SVC_model', SVC(kernel='linear'))
])

# Train model
svm_model_pipeline.fit(X_train, Y_train)

# Predictions
Y_train_pred = svm_model_pipeline.predict(X_train)
Y_test_pred = svm_model_pipeline.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(Y_train, Y_train_pred)
test_accuracy = accuracy_score(Y_test, Y_test_pred)
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", test_accuracy)

# Classification report
print("Classification Report SVM:")
print(classification_report(Y_test, Y_test_pred))

#Save Models
dump(svm_model_pipeline, 'model/email_svm_model.joblib')


Training Accuracy: 0.943125
Validation Accuracy: 0.94
Classification Report SVM:
              precision    recall  f1-score   support

           0       1.00      0.88      0.94       197
           1       0.89      1.00      0.94       203

    accuracy                           0.94       400
   macro avg       0.95      0.94      0.94       400
weighted avg       0.95      0.94      0.94       400



['model/email_svm_model.joblib']

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Pipeline
dt_model_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=max_words, analyzer='char')),
    ('DT_model', DecisionTreeClassifier())
])

# Train model
dt_model_pipeline.fit(X_train, Y_train)

# Predictions
Y_train_pred = dt_model_pipeline.predict(X_train)
Y_test_pred = dt_model_pipeline.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(Y_train, Y_train_pred)
test_accuracy = accuracy_score(Y_test, Y_test_pred)
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", test_accuracy)

# Classification report
print("Classification Report DT:")
print(classification_report(Y_test, Y_test_pred))

#Save Models
dump(dt_model_pipeline, 'model/email_dt_model.joblib')


Training Accuracy: 1.0
Validation Accuracy: 0.9425
Classification Report DT:
              precision    recall  f1-score   support

           0       0.95      0.93      0.94       197
           1       0.94      0.95      0.94       203

    accuracy                           0.94       400
   macro avg       0.94      0.94      0.94       400
weighted avg       0.94      0.94      0.94       400



['model/email_dt_model.joblib']

In [ ]:
# To be Predicted
tobe_predict = ["masharry.net@gmail.com",
                "user@domain",
                "john@test",
                'invalid-email.com',
                "asd.anto@gmail.dfg"]

# Load Model
email_svm_model = load('model/email_svm_model.joblib')
email_dt_model = load('model/email_dt_model.joblib')
email_lr_model = load('model/email_lr_model.joblib')
email_mlp_model = load('model/email_mlp_model.joblib')

#Prediction
print("SVM : ", email_svm_model.predict(tobe_predict))
print("LR : ", email_lr_model.predict(tobe_predict))
print("DT : ", email_dt_model.predict(tobe_predict))
print("MLP : ", email_mlp_model.predict(tobe_predict))

SVM :  [1 0 0 1 1]
LR :  [1 0 0 1 1]
DT :  [0 0 0 1 1]
MLP :  [1 0 0 0 1]
